In [1]:
#import Pkg; Pkg.add("SpeedyWeather")

In [ ]:
using SpeedyWeather
# components
spectral_grid = SpectralGrid(trunc=31, nlayers=8)
output = NetCDFOutput(spectral_grid, PrimitiveWetModel, path="/Users/vidale/SpeedyWeather", id="Climate_Ocean", output_dt=Minute(180))
add!(output, SpeedyWeather.TemperatureOutput(), SpeedyWeather.LargeScalePrecipitationOutput(), SpeedyWeather.ConvectivePrecipitationOutput())  # output also temperature and precip

# Some parameterisation choices
large_scale_condensation = ImplicitCondensation(spectral_grid)
convection = SimplifiedBettsMiller(spectral_grid)
#surface_evaporation = NoSurfaceEvaporation(spectral_grid)
thermodynamics = LandThermodynamics(spectral_grid, heat_conductivity=0.25)
temperature=LandBucketTemperature(spectral_grid)
soil_moisture=LandBucketMoisture(spectral_grid)
land = LandModel(spectral_grid; temperature, soil_moisture)
ocean = SlabOcean(spectral_grid, mixed_layer_depth=50)
#longwave_radiation = NBandRadiation(spectral_grid, nbands=5)

#Try to fix ocean albedo
#albedo = Albedo(AlbedoClimatology(spectral_grid), AlbedoClimatology(spectral_grid))
albedo = Albedo(GlobalConstantAlbedo(spectral_grid, albedo=0.3), AlbedoClimatology(spectral_grid))
# create model, initialize, run
### Entirely new run
#model = PrimitiveWetModel(spectral_grid; large_scale_condensation, convection, land, ocean, albedo, output=output)

### Continuation run
initial_conditions = StartFromFile(path="/Users/vidale/SpeedyWeather", id="0018")   # or specify `path` kwarg if not in this folder
model = PrimitiveWetModel(spectral_grid; initial_conditions, large_scale_condensation, convection, land, ocean, albedo, output=output)

add!(model, SpeedyWeather.AllOutputVariables()...)
simulation = initialize!(model)
run!(simulation, period=Day(30*365), output=true)

[ Info: Time step changed from 2400000 to 2700000 milliseconds (+12%) to match output frequency.
Weather is speedy: run 0022  50%|████▌    |  ETA: 1:02:03 (347.11 years/day)